In [4]:
import pandas as pd
import LEX
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
from numpy.linalg import norm
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
import functions

pd.options.mode.chained_assignment = None  # default='warn'

# FEATURES:

COSINE SIMILARITY OF TF IDF VECTORS OF HEADLINE ARTICLE

NO OF REACTS, NO. OF COMMENTS, NO. OF SHARES

INFORMALITY (NO. OF SWEAR WORDS, NETSPEAK, ETC.) (PROBABLY HAVE TO MAKE FOR THE FILIPINO ONE)

DIVERSITY (NO. OF UNIQUE WORDS, NOUNS, VERBS, ETC.) 

SUBJECTIVITY (NO. OF REPORT VERBS) (HAVE TO MAKE ONE, JUST TRANSALTE THE LIST OF REPORT VERBS AND MAKE CONTER OF THAT)

SENTIMENET (NO. OF EMOTIONAL WORDS) (MAKE ONE)

LENGTH OF SENTENCES (NO OF WORDS PER SENTENCE)

NUMBER OF CHAR PER WORD 

In [5]:
df = pd.read_excel("D:\k//thesis//dataset//sample.xlsx")

In [6]:
df

,Label,Headline,Article,Source,Reactions,Comments,Shares,English,Tagalog,Taglish
0,1,‘Lalaking nagbebenta umano ng expired na pagka...,Arestado ang isang lalaki na naaktuhang nagbeb...,facebook.com/cltv36official/posts/286691113723079,35,4,21,0,1,0
1,1,"INABANDONANG SUGATANG NPA, TINULUNGAN NG MGA K...",Isang pagpapatunay na ang ating mga kasundaluh...,facebook.com/permalink.php?story_fbid=65439452...,6,0,0,0,1,0
2,1,"Amid pandemic, attacks on land rights and live...","Manila, Philippines - The National Federation ...",facebook.com/amihanwomen/posts/1698314000326158,3000,292,3400,1,0,0
3,0,China-backed troll network minamanipula ang op...,Matatandaan na pinaigting ng Facebook ang kani...,facebook.com/FactCheckPhils/posts/353233903685...,3000,315,2700,0,1,0
4,0,"‘Local artists at musicians, nagpalaya ng damd...",Ináua o breath — ito ang naging tema ng intera...,facebook.com/cltv36news/posts/295249866181290,30,5,35,0,1,0


In [7]:
df_t = df.loc[df['Tagalog']==1]

In [8]:
df_e = df.loc[df['English']==1]

In [9]:
df_t['Cosine']=df_t.apply(lambda x: functions.getcosine(df_t['Headline'].values[0],df_t['Article'].values[0]), axis = 1)

In [10]:
df_t['N_TR'] = df_t['Article'].apply(LEX.noun_token_ratio)

In [11]:
df_t['V_TR'] = df_t['Article'].apply(LEX.verb_token_ratio)

In [12]:
df_t['TTR'] = df_t['Article'].apply(LEX.type_token_ratio)

In [13]:
df_t['word_per_sen'] = df_t['Article'].apply(functions.wordpsen)

In [14]:
df_t['char_per_word'] = df_t['Article'].apply(functions.charpword)

In [15]:
df_t

,Label,Headline,Article,Source,Reactions,Comments,Shares,English,Tagalog,Taglish,Cosine,N_TR,V_TR,TTR,word_per_sen,char_per_word
0,1,‘Lalaking nagbebenta umano ng expired na pagka...,Arestado ang isang lalaki na naaktuhang nagbeb...,facebook.com/cltv36official/posts/286691113723079,35,4,21,0,1,0,0.0,0.223881,0.097015,0.179104,20.375000,4.607362
1,1,"INABANDONANG SUGATANG NPA, TINULUNGAN NG MGA K...",Isang pagpapatunay na ang ating mga kasundaluh...,facebook.com/permalink.php?story_fbid=65439452...,6,0,0,0,1,0,0.0,0.316206,0.086957,0.079051,34.625000,4.951264
3,0,China-backed troll network minamanipula ang op...,Matatandaan na pinaigting ng Facebook ang kani...,facebook.com/FactCheckPhils/posts/353233903685...,3000,315,2700,0,1,0,0.0,0.205882,0.102941,0.137255,32.142857,5.280000
4,0,"‘Local artists at musicians, nagpalaya ng damd...",Ináua o breath — ito ang naging tema ng intera...,facebook.com/cltv36news/posts/295249866181290,30,5,35,0,1,0,0.0,0.313808,0.066946,0.117155,20.230769,4.942966


In [ ]:
df_t

# ah hell nah tf-idf

In [55]:
art = df_t.loc[df_t.index==0]['Article'].values[0]
head = df_t.loc[df_t.index==0]['Headline'].values[0]

In [56]:
art_tokens = nltk.tokenize.word_tokenize(art)
head_tokens = nltk.tokenize.word_tokenize(head)

In [65]:
un_tokens = set(art_tokens).union(set(head_tokens))

In [66]:
num_art_words = dict.fromkeys(un_tokens,0)
for word in art_tokens:
#     print(word)
    num_art_words[word] += 1
num_head_words = dict.fromkeys(un_tokens,0)
for word in head_tokens:
#     print(word)
    num_head_words[word] += 1

In [67]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [68]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [73]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [75]:
tf_art = computeTF(num_art_words, art_tokens)
tf_head = computeTF(num_head_words, head_tokens)

In [70]:
idf = computeIDF([num_art_words,num_head_words])

In [76]:
tfidf_art = computeTFIDF(tf_art, idf)
tfidf_head = computeTFIDF(tf_head, idf)

# cosine similarity

In [109]:
art_vect = np.array([*tfidf_head.values()])
head_vect = np.array([*tfidf_art.values()])

In [110]:
cosine = np.dot(head_vect,art_vect) / (norm(head_vect)*norm(art_vect))

In [101]:
# other way to get cosine
# spatial.distance.cosine(head_vect,art_vect)

# art_vect = art_vect.reshape(1,-1)
# head_vect = head_vect.reshape(1,-1)

# cosine_similarity(head_vect,art_vect)

1.0

In [96]:
cosine

0.0

In [1]:
import functions

In [2]:
functions.charpword

<function functions.charpword(art)>

In [174]:
fcts.ordpsen()

AttributeError: module 'functions' has no attribute 'ordpsen'

In [134]:
nltk.sent_tokenize(df_t.loc[df_t.index == 0]['Article'].values[0])

['Arestado ang isang lalaki na naaktuhang nagbebenta ng expired umanong mga pagkain sa Brgy.',
 'Colgante, Apalit, Pampanga.',
 'Isinagawa ang buy-bust operation nitong Martes, June 27.',
 'Nasamsam mula sa suspek ang mga expired na produktong binebenta kagaya ng 4 na kahon ng imported chocolates, 4 na kahon ng local cheese, 21 kahon ng cup noodles, 25 pakete ng biscuits, 100 pakete ng powder juice, at 2 kahon na cheese spread.',
 'Sa kabuuan, tinatayang nagkakahalaga ang mga ito ng ₱30,000.',
 'Iniimbestigahan na ng pulisya ang pinanggalingan ng expired products at tugisin ang mastermind.',
 'Lubha umanong mapanganib sa kalusugan ng mga tao ang ganitong uri ng pagkain kaya babala nila sa publiko na tingnan muna ang expiration date ng anumang produktong bibilhin lalo na kung ito ay inaalok lamang sa mga kalsada.',
 'Samantala, sasampahan ang suspek ng kasong paglabag sa RA 7394 o Consumer Act of the Philippines.']

In [4]:
test_art = df.loc[df.index == 5]

In [5]:
art_str = test_art['article'].values[0]

In [14]:
# import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karlo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
LEX.type_token_ratio(art_str)

0.4146341463414634